In [1]:
#%matplotlib inline
%matplotlib widget

In [2]:
# initialization
from rayoptics.environment import *

from rayoptics.optical import analyses
from rayoptics.mpl.analysisfigure import Wavefront, RayFanPlot, AnalysisFigure

from matplotlib import gridspec

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [3]:
root_pth = Path(rayoptics.__file__).resolve().parent

# Create a new model

In [4]:
opm = OpticalModel()
sm = opm.seq_model
osp = opm.optical_spec
pm = opm.parax_model

## Define first order aperture and field for system

In [5]:
pupil_diameter = 100.
pupil_radius = pupil_diameter/2
osp.pupil = PupilSpec(osp, key=['object', 'pupil'], value=pupil_diameter)

# single field on-axis
osp.field_of_view = FieldSpec(osp, key=['object', 'angle'], flds=[0.0])

# wavelength for analysis: 550nm
osp.spectral_region = WvlSpec([(550.0, 1.0)], ref_wl=0)

### object at infinity, i.e. collimated input

In [6]:
sm.gaps[0].thi = 1e+11

In [7]:
#opm.add_mirror(lbl='M1', r=-500., t=-250.)
opm.add_mirror(lbl='M1', profile=Conic, r=-500., cc=-1., t=-250.)

In [8]:
opm.update_model()
fod = osp.parax_data.fod

# Draw a lens picture

In [9]:
layout_plt = plt.figure(FigureClass=InteractiveLayout, opt_model=opm,
                        do_draw_rays=True, do_paraxial_layout=False).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# List first order data

In [10]:
fod.list_first_order_data()

efl                 250
ffl                -250
pp1                  -0
bfl                 250
ppk                   0
f/#                 2.5
m                     0
red              -4e+08
obj_dist          1e+11
obj_ang               1
enp_dist             -0
enp_radius           50
na obj            5e-10
n obj                 1
img_dist           -250
img_ht            4.364
exp_dist             -0
exp_radius           50
na img           0.1961
n img                -1
optical invariant       0.8728


In [11]:
fod.img_na

0.196116134666751

In [12]:
opd = 0.25*opm.nm_to_sys_units(osp.spectral_region.central_wvl)
dfoc = opd/(fod.img_na**2/(2*fod.n_img)); dfoc

-0.0071500000343749995

In [13]:
fld, wvl, foc = osp.lookup_fld_wvl_focus(0)

In [14]:
osp.defocus.focus_shift = dfoc/2

### create fan and grid objects for use by plot grid

In [15]:
ray_xfan = analyses.RayFan(opm, f=fld, wl=wvl, xyfan='x')
ray_yfan = analyses.RayFan(opm, f=fld, wl=wvl, xyfan='y')
ray_grid = analyses.RayGrid(opm, f=fld, wl=wvl)

Create lists of fans, data types, and plotting keyword arguments to drive 

In [16]:
xyabr_fan_list = [(ray_xfan, 'dx', dict(num_points=100)),
                  (ray_yfan, 'dy', dict(num_points=100, linestyle='--'))]

In [17]:
opd_fan_list = [(ray_yfan, 'opd', dict(linestyle='', linewidth=1, marker='D', markersize=3.5)),
                (ray_yfan, 'opd', dict(num_points=100, linewidth=2))]

### create focus dashboard

In [18]:
def create_focus_dashboard(fig, opm, fld, wvl):
    osp = opm.optical_spec
    fod = osp.parax_data.fod
    foc = osp.defocus.focus_shift
    opd = opm.nm_to_sys_units(wvl)
    # one wave of defocus
    dfoc = opd/(fod.img_na**2/(2*fod.n_img))
    qwrt_dfoc = abs(0.25*dfoc)
    # one wave of tilt
    _, _, ref_sphere_radius = fld.ref_sphere
    shft = ref_sphere_radius*opd/fod.exp_radius
    defocus=widgets.FloatSlider(min=-qwrt_dfoc, max=+qwrt_dfoc, step=.05*qwrt_dfoc,
                                description='defocus', value=foc,
                                readout_format='.4f', continuous_update=True)
    x_shift=widgets.FloatSlider(min=-1.5*shft, max=+1.5*shft, step=.05*shft,
                                description='x shift', value=0.,
                                readout_format='.4f', continuous_update=True)
    y_shift=widgets.FloatSlider(min=-1.5*shft, max=+1.5*shft, step=.05*shft,
                                description='y shift', value=0.,
                                readout_format='.4f', continuous_update=True)

    def slider_update(change):
        dfoc_val = defocus.value
        dx = x_shift.value
        dy = y_shift.value

        # apply changes to fans and grids
        for data_obj in fig.data_objs:
            data_obj.foc = dfoc_val
            data_obj.image_pt_2d = np.array([dx, dy])

        # update and plot results
        fig.refresh(build='update')
        fig2.refresh(build='update')

    defocus.observe(slider_update, names='value')
    x_shift.observe(slider_update, names='value')
    y_shift.observe(slider_update, names='value')
    
    return defocus, x_shift, y_shift

In [19]:
fig = plt.figure(FigureClass=AnalysisFigure, data_objs=[ray_grid, ray_xfan, ray_yfan],
                 figsize=[9, 5], tight_layout=True)
gs = gridspec.GridSpec(nrows=8, ncols=13, figure=fig)

Wavefront(fig, gs[:8, :8], ray_grid, user_scale_value=1.5, do_contours=False, title='Wavefront Map')
RayFanPlot(fig, gs[:4, 9:], xyabr_fan_list, user_scale_value=0.001, scale_type='user',
           yaxis_ticks_position='right', title='Transverse Ray Aberration')
RayFanPlot(fig, gs[4:8, 9:], opd_fan_list, user_scale_value=1.5, scale_type='user',
           yaxis_ticks_position='right', title='Wavefront Aberration')
fig.refresh()
defocus, x_shift, y_shift = create_focus_dashboard(fig, opm, fld, wvl)
display(widgets.HBox([defocus, y_shift]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig2 = plt.figure(FigureClass=AnalysisFigure, data_objs=[ray_xfan, ray_yfan],
                 figsize=[9, 5])
gs2 = gridspec.GridSpec(nrows=1, ncols=2, figure=fig2)

RayFanPlot(fig2, gs2[0, 0], xyabr_fan_list, user_scale_value=0.001, scale_type='user',
           title='Transverse Ray Aberration')
RayFanPlot(fig2, gs2[0, 1], opd_fan_list, user_scale_value=1.5, scale_type='user',
           yaxis_ticks_position='right', title='Wavefront Aberration')
fig2.refresh()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig.clf()
fig.refresh(build='update')

In [22]:
4*opd

0.0005499999999999999

In [23]:
fod.exp_radius

50.0

In [24]:
tan_a = 4*opd/fod.exp_radius; tan_a

1.0999999999999998e-05

In [25]:
fld.ref_sphere[2]

250.00357500001718

In [26]:
1.5*tan_a*fld.ref_sphere[2]

0.004125058987500283